In [9]:
import pandas as pd
import pymssql
import pymysql
import sys
import hashlib
sys.path.append('/Users/yuming/Github/helios/helios/app')
from common.log import EXCEPT, set_log_cate, def_log_cates, cust_log_cates
import utils.log as logging
from common.nlp.word_normalizer import Normalizer

In [6]:
CONN_FUNC = {'mysql': pymysql.connect, 'mssql': pymssql.connect}

SERVER = {'mysql': {'ip': '192.168.7.74',
                    'username': 'root',
                    'pwd': 'Helios123',
                    'port': 3306},
          'mssql': {'ip': '192.168.7.155',
                    'username': 'SA',
                    'pwd': 'Helios123',
                    'port': 1433}
          }
logging.setup(logfile='/Users/yuming/log/form_norm.log')
set_log_cate(def_log_cates | cust_log_cates)

In [3]:
sql = 'mssql'
dbname = 'hr_pii'
conn = CONN_FUNC.get(sql)(SERVER[sql]['ip'], SERVER[sql]['username'], SERVER[sql]['pwd'], dbname)

In [4]:
df = pd.read_sql('SELECT * FROM employee', con=conn)

In [7]:
file_dict = df.to_dict('records')
output_list = []
for record in file_dict:
    normalizer = Normalizer(record, 'en')
    normalizer.action()
    output_list.append(normalizer.dump())
df_norm = pd.DataFrame(output_list)

In [8]:
df_norm.head()

,birthdate,blood_group,ethnic,health,mail,name,phone,politics,race,religion,sex,sexual,ssn,trade_union
0,1992-04-17,ab+,hispanic or latino or spanish origin,disabled,juliakaufman@hotmail.com,wayne jones,(802)111-7264,isempty,white,caodaism,male,homosexual,001828525,isempty
1,1954-11-24,ab+,isempty,isempty,hallerika@gmail.com,christopher burgess,(731)673-6998,isempty,isempty,isempty,male,isempty,005252021,isempty
2,1961-09-28,ab+,isempty,isempty,fdurham@hotmail.com,yvette ramsey,(731)305-6279,isempty,isempty,isempty,female,isempty,005324497,isempty
3,1977-01-03,b+,hispanic or latino or spanish origin,isempty,christine44@hotmail.com,john frank,(215)986-2896,democratic party,white,isempty,male,asexual,007412966,false
4,1992-09-13,b-,isempty,isempty,rthomas@hotmail.com,heather rogers,(405)889-4448,isempty,isempty,isempty,female,isempty,010995324,isempty


In [18]:
def get_md5(string):
    string = string.encode('utf-8')
    return {string: hashlib.md5(string).hexdigest()}

In [19]:
df = df_norm.applymap(get_md5)

In [21]:
df.to_csv('employee_md5.csv')